#Python Tutorial With Google Colab

This tutorial was originally written by [Justin Johnson](https://web.eecs.umich.edu/~justincj/) for cs231n. It was adapted as a Jupyter notebook for Stanford cs228 by [Volodymyr Kuleshov](http://web.stanford.edu/~kuleshov/) and [Isaac Caswell](https://symsys.stanford.edu/viewing/symsysaffiliate/21335).

This version has been adapted for Colab by Kevin Zakka for the Spring 2020 edition of [cs231n](https://cs231n.github.io/). It runs Python3 by default.

I have made some modification to this tutorial.

##Introduction

Python is a great general-purpose programming language on its own, but with the help of a few popular libraries (numpy, scipy, matplotlib) it becomes a powerful environment for scientific computing.

We expect that many of you will have some experience with Python and numpy; for the rest of you, this section will serve as a quick crash course both on the Python programming language and on the use of Python for scientific computing.

Some of you may have previous knowledge in Matlab, in which case we also recommend the numpy for Matlab users page (https://docs.scipy.org/doc/numpy-dev/user/numpy-for-matlab-users.html).

In this tutorial, we will cover:

* Basic Python: Basic data types (Containers, Lists, Dictionaries, Sets, Tuples), Functions, Classes
* Numpy: Arrays, Array indexing, Datatypes, Array math, Broadcasting
* Matplotlib: Plotting, Subplots, Images
* IPython: Creating notebooks, Typical workflows

## A Brief Note on Python Versions

As of Janurary 1, 2020, Python has [officially dropped support](https://www.python.org/doc/sunset-python-2/) for `python2`. We'll be using Python 3.7 for this iteration of the course. You can check your Python version at the command line by running `python --version` or `python3 --version`. In Colab, we can enforce the Python version by clicking `Runtime -> Change Runtime Type` and selecting `python3`. Note that as of April 2020, Colab uses Python 3.6.9 which should run everything without any errors.

In [5]:
!python --version

Python 3.9.16


In [6]:
!ls sample_data

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


##Basics of Python

Python is a high-level, dynamically typed multiparadigm programming language. Python code is often said to be almost like pseudocode, since it allows you to express very powerful ideas in very few lines of code while being very readable. As an example, here is an implementation of the classic quicksort algorithm in Python:

In [7]:
def quicksort(arr):
    if len(arr) <= 1: 
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)

print(quicksort([3,6,8,10,1,2,1]))

[1, 1, 2, 3, 6, 8, 10]


###Basic data types

Simple data types in Python:
* **int** (integers): numbers without a decimal
* **float** (floating point numbers): numbers with a decimal
* **bool** (booleans): True or False values
* **str** (strings): usually to represent text. However, anything that is wrapped in quotes (single or double) is treated as a string.

####Numbers

Integers and floats work as you would expect from other languages:

In [8]:
x = 3
print(x, type(x))

3 <class 'int'>


In [9]:
x, y, z = 1, 2, 3  # Multiple assignment 
print(x, y, z)

1 2 3


In [10]:
print(x + 1)   # Addition
print(x - 1)   # Subtraction
print(x * 2)   # Multiplication
print(x ** 2)  # Exponentiation

2
0
2
1


In [11]:
x += 1    # x = x + 1 
print(x)
x *= 2    # x = x * 2
print(x)

2
4


In [12]:
y = 2.5
print(type(y))
print(y, y + 1, y * 2, y ** 2)

<class 'float'>
2.5 3.5 5.0 6.25


Note that unlike many languages, **Python does not have unary increment** (x++) or decrement (x--) operators.

Python also has built-in types for long integers and complex numbers; you can find all of the details in the [documentation](https://docs.python.org/3.7/library/stdtypes.html#numeric-types-int-float-long-complex).

####Booleans

Python implements all of the usual operators for Boolean logic, but uses English words rather than symbols (`&&`, `||`, etc.):

In [13]:
t, f = True, False   # Mulitple assignments
print(type(t))

<class 'bool'>


Now we let's look at the operations:

In [14]:
print(t and f) # Logical AND;
print(t or f)  # Logical OR;
print(not t)   # Logical NOT;
print(t != f)  # Logical XOR;

False
True
False
True


####Strings

In [15]:
hello = 'hello'   # String literals can use single quotes
world = "world"   # or double quotes; it does not matter
print(hello)
print(len(hello))

hello
5


In [16]:
hw = hello + ', ' + world  # String concatenation
print(hw)

hello, world


In [17]:
hw12 = '{} {} {}'.format(hello, world, 12)  # string formatting
print(hw12)

hello world 12


In [18]:
hw12 = hello + ' ' + world + ' ' + str(3.1415)
print(hw12)

hello world 3.1415


String objects have a bunch of useful methods; for example:

In [19]:
s = "hello"
print(s.capitalize())  # Capitalize a string
print(s.upper())       # Convert a string to uppercase; prints "HELLO"
print(s.rjust(7))      # Right-justify a string, padding with spaces
print(s.center(7))     # Center a string, padding with spaces
print(s.replace('l', '(ell)'))  # Replace all instances of one substring with another

Hello
HELLO
  hello
 hello 
he(ell)(ell)o


In [20]:
s = 'hello'
s = s.upper()
print(s)

HELLO


**Whitespace**: any nonprinting character, such as spaces, tabs, and end-of-line symbols

In [21]:
print("ECE 498/598\n\tDeep Learning\nis fun!")

ECE 498/598
	Deep Learning
is fun!


In [22]:
print('  w\to\tr  l\nd '.strip())  # Strip leading and trailing whitespace

w	o	r  l
d


In [23]:
print('  world '.rstrip())  # Strip trailing whitespace

  world


In [24]:
print('  world '.lstrip())  # Strip leading whitespace

world 


You can find a list of all string methods in the [documentation](https://docs.python.org/3.7/library/stdtypes.html#string-methods).

###Containers

Python includes several built-in container types: lists, dictionaries, sets, and tuples.

####Lists

A list is the Python equivalent of an array, but is resizeable and can contain elements of different types:

In [25]:
xs = [1, 2, 3, 'hello', [4, 5, 6]]    # Create a list
print(xs)
print('First element of the array: ', xs[0])  # Index to the list starts with 0
print('Last element of the array: ', xs[4])

[1, 2, 3, 'hello', [4, 5, 6]]
First element of the array:  1
Last element of the array:  [4, 5, 6]


In [26]:
# Negative indices count from the end of the list; 
print(xs[-1])     # Index -1 returns the last item in the list
print(xs[-2])     # Index -2 returns the second item from the end of the list

[4, 5, 6]
hello


In [27]:
xs[2] = 'foo'    # Lists can contain elements of different types
print(xs)

[1, 2, 'foo', 'hello', [4, 5, 6]]


In [28]:
xs.append('bar') # Add a new element to the end of the list
print(xs)  

[1, 2, 'foo', 'hello', [4, 5, 6], 'bar']


In [29]:
print(xs)
x = xs.pop()     # Remove and return the last element of the list
print(x)
print(xs)

[1, 2, 'foo', 'hello', [4, 5, 6], 'bar']
bar
[1, 2, 'foo', 'hello', [4, 5, 6]]


In [30]:
xs.insert(1, "new item") # Insert the "new item" at the index 1
print(xs) 

[1, 'new item', 2, 'foo', 'hello', [4, 5, 6]]


In [31]:
xs.remove("foo") # Removing an item by value
print(xs)

[1, 'new item', 2, 'hello', [4, 5, 6]]


As usual, you can find all the gory details about lists in the [documentation](https://docs.python.org/3.7/tutorial/datastructures.html#more-on-lists).

####Slicing

In addition to accessing list elements one at a time, Python provides concise syntax to access sublists; this is known as slicing:

In [32]:
nums = list(range(5))    # range is a built-in function that creates a list of integers
nums = [0, 1, 2, 3, 4]
print(nums)         # Prints "[0, 1, 2, 3, 4]"
print(nums[2:4])    # Get a slice from index 2 to 4 (exclusive); prints "[2, 3]"
print(nums[2:])     # Get a slice from index 2 to the end; prints "[2, 3, 4]"
print(nums[:2])     # Get a slice from the start to index 2 (exclusive); prints "[0, 1]"
print(nums[:])      # Get a slice of the whole list; prints ["0, 1, 2, 3, 4]"
print(nums[:-1])    # Slice indices can be negative; prints ["0, 1, 2, 3]"
nums[2:4] = [8, 9]  # Assign a new sublist to a slice
print(nums)         # Prints "[0, 1, 8, 9, 4]"

[0, 1, 2, 3, 4]
[2, 3]
[2, 3, 4]
[0, 1]
[0, 1, 2, 3, 4]
[0, 1, 2, 3]
[0, 1, 8, 9, 4]


####Loops

You can loop over the elements of a list like this:

In [33]:
animals = ['cat', 'dog', 'monkey']
for animal in animals:
  print(animal)

cat
dog
monkey


If you want access to the index of each element within the body of a loop, use the built-in `enumerate` function:

In [34]:
animals = ['cat', 'dog', 'monkey']
for idx, animal in enumerate(animals):
    print('#{}: {}'.format(idx, animal))

#0: cat
#1: dog
#2: monkey


#### Copying a list

To copy a list, you can make a slice that includes the entire original list
by omitting the first index and the second index ([:]). This tells Python to
make a slice that starts at the first item and ends with the last item, producing
a copy of the entire list.

In [35]:
a = [1, 2, 3, 4, 5]
b = a[:]  # copying the list
# b = a.copy()
c = a     # This doesn't copy the list!

a.append('x') # a = [1, 2, 3, 4, 5, 'x']
b.append('y') # b = [1, 2, 3, 4, 5, 'y']
c.append('z') # c = [1, 2, 3, 4, 5, 'x', 'z']

print('a = ', a)
print('b = ', b)
print('c = ', c)
print('a = ', a)

a =  [1, 2, 3, 4, 5, 'x', 'z']
b =  [1, 2, 3, 4, 5, 'y']
c =  [1, 2, 3, 4, 5, 'x', 'z']
a =  [1, 2, 3, 4, 5, 'x', 'z']


####List comprehensions:

When programming, frequently we want to transform one type of data into another. As a simple example, consider the following code that computes square numbers:

In [36]:
nums = [0, 1, 2, 3, 4]
squares = []
for x in nums:
    squares.append(x ** 2)
print(squares)

[0, 1, 4, 9, 16]


You can make this code simpler using a list comprehension:

In [37]:
nums = [0, 1, 2, 3, 4]
squares = [x ** 2 for x in nums]
print(squares)

[0, 1, 4, 9, 16]


List comprehensions can also contain conditions:

In [38]:
nums = [0, 1, 2, 3, 4]
even_squares = [x ** 2 for x in nums if (x % 2 == 0 and x <=2)]
print(even_squares)

[0, 4]


####Dictionaries

A dictionary stores (key, value) pairs, similar to a `Map` in Java or an object in Javascript. You can use it like this:

In [39]:
d = {'cat': 'cute', 'dog': 'furry'}  # Create a new dictionary with some data
print(d['cat'])       # Get an entry from a dictionary; prints "cute"
key = 'cat'
print(d[key])
print('cat' in d)     # Check if a dictionary has a given key; prints "True"

cute
cute
True


Adding a new key-value pair

In [40]:
d['fish'] = 'wet'    # Set an entry in a dictionary
print(d['fish'])     # Prints "wet"
d['elephant'] = 'heavy'  # Set an entry in a dictionary
print(d['elephant'])     # Prints "heavy"

wet
heavy


Looping through a dictionary

In [41]:
for key, value in d.items():
  print('key = ', key, '\t value = ', value)

key =  cat 	 value =  cute
key =  dog 	 value =  furry
key =  fish 	 value =  wet
key =  elephant 	 value =  heavy


In [42]:
for key in d.keys():
  print('key = ', key, '\tvalue = ', d[key])

key =  cat 	value =  cute
key =  dog 	value =  furry
key =  fish 	value =  wet
key =  elephant 	value =  heavy


In [43]:
for key in sorted(d.keys()):  # sorting the key
  print('key = ', key, '\t\tvalue = ', d[key])

key =  cat 		value =  cute
key =  dog 		value =  furry
key =  elephant 		value =  heavy
key =  fish 		value =  wet


In [44]:
for value in sorted(d.values()):  # print all values
  print('value = ', value)

value =  cute
value =  furry
value =  heavy
value =  wet


In [45]:
print('monkey' in d)
print(d['monkey'])  # KeyError: 'monkey' not a key of d

False


KeyError: ignored

For dictionaries, we can use the get() method to set a default value that will be returned if the requested key doesn’t exist.

In [ ]:
print(d.get('monkey', 'N/A'))  # Get an element with a default; prints "N/A"
print(d.get('fish', 'N/A'))    # Get an element with a default; prints "wet"

Removing a key-value pair

In [ ]:
del d['fish']        # Remove an element from a dictionary

# Be aware that the deleted key-value pair is removed permanently.
print(d.get('fish', 'N/A')) # "fish" is no longer a key; prints "N/A"

You can find all you need to know about dictionaries in the [documentation](https://docs.python.org/2/library/stdtypes.html#dict).

It is easy to iterate over the keys in a dictionary:

In [ ]:
d = {'person': 2, 'cat': 4, 'spider': 8}
for animal, legs in d.items():
    print('A {} has {} legs'.format(animal, legs))

Dictionary comprehensions: These are similar to list comprehensions, but allow you to easily construct dictionaries. For example:

In [ ]:
nums = [0, 1, 2, 3, 4]
even_num_to_square = {x: x ** 2 for x in nums if x % 2 == 0}
print(even_num_to_square)

####Sets

A set is an unordered collection of distinct elements. As a simple example, consider the following:

In [ ]:
animals = {'cat', 'dog', 'cat'}
print(len(animals))       
print('cat' in animals)   # Check if an element is in a set; prints "True"
print('fish' in animals)  # prints "False"


In [ ]:
animals.add('fish')      # Add an element to a set
print('fish' in animals)
print(len(animals))       # Number of elements in a set;

In [ ]:
animals.add('cat')       # Adding an element that is already in the set does nothing
print(len(animals))       
animals.remove('cat')    # Remove an element from a set
print(len(animals))       

_Loops_: Iterating over a set has the same syntax as iterating over a list; however since sets are unordered, you cannot make assumptions about the order in which you visit the elements of the set:

In [ ]:
animals = {'cat', 'dog', 'fish'}
for idx, animal in enumerate(animals):
    print('#{}: {}'.format(idx, animal))

Set comprehensions: Like lists and dictionaries, we can easily construct sets using set comprehensions:

In [ ]:
from math import sqrt
print({int(sqrt(x)) for x in range(30)})

####Tuples

A tuple is an (immutable) ordered list of values. Python refers to values that cannot change as immutable, and an immutable list is called a tuple.

In [ ]:
dimensions = (800, 600)
print(dimensions[0])
print(dimensions[1])

In [ ]:
dimensions[0] = 1000  # this will return error 

A tuple is in many ways similar to a list; one of the most important differences is that tuples can be used as keys in dictionaries and as elements of sets, while lists cannot. Here is a trivial example:

In [ ]:
d = {(x, x + 1): x for x in range(10)}  # Create a dictionary with tuple keys
t = (5, 6)       # Create a tuple
print(type(t))
print(d[t])       
print(d[(1, 2)])

In [ ]:
t[0] = 1

###Functions

Python functions are defined using the `def` keyword. For example:

In [ ]:
def sign(x, y=100):
    if x > 0:
        return 'positive', 123
    elif x < 0:
        return 'negative', 123
    else:
        return 'zero', 123

for x in [-1, 0, 1]:
    v1, _ = sign(x)
    print(v1)

We will often define functions to take optional keyword arguments, like this:

In [ ]:
def hello(name, loud=False):
    if loud:
        print('HELLO, {}'.format(name.upper()))
    else:
        print('Hello, {}!'.format(name))

hello('Bob')
hello('Fred', loud=True)

Modifing a list in a function

When you pass a list to a function, the function can modify the list.

In [ ]:
def fun(list):
  list.append('new item')

a = [1, 2, 3]
fun(a)
print(a)

If you want to prevent a function from modifying a list, you can send a copy of a list to the function

In [ ]:
def fun(list):
  list.append('new item')

a = [1, 2, 3]
fun(a[:]) # The slice notation [:] makes a copy of the list
# fun(a.copy()) 
print(a)

###Classes

#### Defining a Class

The syntax for defining classes in Python is straightforward:

In [ ]:
class Addition: 
	# first = 0
	# second = 0
	answer = 0
	
	# parameterized constructor 
	def __init__(self, first, second): 
		self.first = first 
		self.second = second 

	def display(self): 
		print("First number = " + str(self.first)) 
		print("Second number = " + str(self.second)) 
		print("Addition of two numbers = " + str(self.answer)) 

	def calculate(self): 
		self.answer = self.first + self.second 
	
# creating object of the class 
# this will invoke parameterized constructor 
obj = Addition(1000, 2000) 

# perform Addition 
obj.calculate() 

# display result 
obj.display() 


In [ ]:
class Person:

    # Constructor
    def __init__(self, name):
        self.name = name  # Create an instance variable

    # Instance method
    def getName(self):
        return self.name 

    # To check if this person is an employee 
    def isEmployee(self): 
        return False

# Making an instance from a class
g = Person('Fred')  # Construct an instance of the Greeter class

# Accessing attributes
print(g.name)

# Calling methods
print(g.getName())          

* The **self** parameter is required in the method definition, and it must come first before the other parameters. It must be included in the definition because when Python calls this method later (to create an instance), the method call will automatically pass the self argument.
* Every method call associated with an instance automatically passes **self**, which is a reference to the instance itself; it gives the individual instance access to the attributes and methods in the class.

In a class, the implicit behavior of passing the object as the first argument is avoided if a method is declared as static, as shown in the following example.

In [ ]:
class A(object):

    @staticmethod
    def stat_meth():
        print("Look no self was passed")

a = A()
a.stat_meth()

#### Inheritance

Inheritance is the capability of one class to derive or inherit the properties from another class. The benefits of inheritance are:

* It represents real-world relationships well.
* It provides reusability of a code. We don’t have to write the same code again and again. Also, it allows us to add more features to a class without modifying it.
* It is transitive in nature, which means that if class B inherits from another class A, then all the subclasses of B would automatically inherit from class A.

In [ ]:
# Inherited or Subclass (Note Person in bracket) 
class Employee(Person): 
   
    # Constructor
    def __init__(self, name, year):
        self.years = year  # Create an instance variable
        super().__init__(name)

    # Overriding Methods from the Parent Class
    def isEmployee(self): 
        return True
   
# Driver code 
emp = Person("Geek1")  # An Object of Person 
print(emp.getName(), emp.isEmployee()) 
   
emp = Employee("Geek2", 20) # An Object of Employee 
print(emp.getName(), emp.isEmployee()) 

## Iterators

#### An iterator is an object that can be iterated upon, meaning that you can traverse through all the values.

In [ ]:
mytuple = ("apple", "banana", "cherry")
myit = iter(mytuple) # define

print(next(myit)) # apple
print(next(myit)) # banana
print(next(myit)) # cherry
print(next(myit)) # error

#### Strings, lists, tuples, dictionaries, and sets are all iterable objects.

In [ ]:
mystr = "banana"
myit = iter(mystr)

print(next(myit))
print(next(myit))
print(next(myit))
print(next(myit))
print(next(myit))
print(next(myit))

#### Looping Through an Iterator

C-style approach

In [ ]:
mytuple = ("apple", "banana", "cherry")
i = 0
while (i < len(mytuple)): 
    print(mytuple[i])
    i += 1

A better approach

In [ ]:
mytuple = ("apple", "banana", "cherry")

print(mytuple)

for idx, val in enumerate(mytuple):
  print(idx, val)

In [ ]:
mystr = "banana"

for x in mystr:
  print(x)

In [ ]:
# Iterating over dictionary 
d = dict()  
d['xyz'] = 123
d['abc'] = 345
for i in d : 
    # print("%s  %d" %(i, d[i]))
    print("{}\t{}".format(i, d[i]))

#### Create an Iterator


To create an object/class as an iterator, you have to implement the methods \_\_iter\_\_() and \_\_next\_\_() to your object.


*   \_\_iter\_\_ method that is called on initialization of an iterator. This should return an object that has a \_\_next\_\_ method.
*   \_\_next\_\_ should return the next value for the iterable. This method should raise a **StopIteration** to signal the end of the iteration.



In [ ]:
class MyNumbers:
  def __iter__(self):
    self.a = 1
    return self

  def __next__(self):
    x = self.a
    self.a += 1
    return x

myclass = MyNumbers()
myiter = iter(myclass)

print(next(myiter))
print(next(myiter))
print(next(myiter))
print(next(myiter))
print(next(myiter))

#for x in myiter:
#  print(x)


Stop after 10 iterations by using the StopIteration statement

In [ ]:
class MyNumbers:
  def __iter__(self):
    self.a = 1
    return self

  def __next__(self):
    if self.a <= 10:
      x = self.a
      self.a += 1
      return x
    else:
      raise StopIteration

myclass = MyNumbers()
myiter = iter(myclass)

for x in myiter:
  print(x)

#### zip function
Combine mulitple iterator

In [ ]:
# Two separate lists 
thing = ["Apple", "Audi", "Pasta", "dog", "UMAINE"] 
category = ["fruit", "car",  "food", "animal"] 
  
# Combining lists and printing 
for t, c in zip(thing, category): 
    print("{} is {}".format(t, c))

Use "*" operator to unzip

In [ ]:
l1,l2 = zip(*[('Apple', 'fruit'),  
              ('Audi', 'car'),  
              ('Pasta', 'food')  
           ]) 
  
# Printing unzipped lists       
print(l1) 
print(l2) 

## yield vs return

The yield statement suspends function’s execution and sends a value back to caller, but retains enough state to enable function to resume where it is left off. When resumed, the function continues execution immediately after the last yield run. This allows its code to produce a series of values over time, rather them computing them at once and sending them back like a list.

In [ ]:
def simpleGeneratorFun(): 
    x = 5
    yield 1 + x
    x = 2*x
    yield 2 + x
    print(x)
    x = 2*x
    print(x)
    yield 3 + x

# Driver code to check above generator function 
# simpleGeneratorFun()
list(simpleGeneratorFun())

#for value in simpleGeneratorFun():  
#    print(value) 

Yield are used in Python generators. A generator function is defined like a normal function, but whenever it needs to generate a value, it does so with the yield keyword rather than return. If the body of a def contains yield, the function automatically becomes a generator function.

In [ ]:
def nextSquare(): 
    i = 1; 
  
    # An Infinite loop to generate squares  
    while True: 
        yield i*i                 
        i += 1  # Next execution resumes  
                # from this point      
  
# Driver code to test above generator  
# function 
for num in nextSquare(): 
    if num > 100: 
         break    
    print(num) 

## Modules and Packages

In [ ]:
!pip install import-ipynb

In [ ]:
import import_ipynb
import myModule

print(myModule.s)


In [ ]:
import sys
import os

dirpath = os.getcwd()
print("current directory is : " + dirpath)
foldername = os.path.basename(dirpath)
print("Directory name is : " + foldername)